In [1]:
from tda.client import Client
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime

import seaborn as sns

import login.config as config
from login.client import TDSession

In [2]:
#1 min = 60000ms
def mins_to_ms(mins):
    return mins*60000

#function to get the period desired
def get_period(mins, from_time, df):
    start = df['datetime']> from_time
    end = df['datetime']<=(from_time+mins_to_ms(mins))
    return df.loc[start&end,['datetime','low']]

#loop through low price of period to check for sell
def did_sell(df_period, sell_price,i):
    return [row1 if row1 >= sell_price else False for row1 in df_period]

#loop through low price of period to check for sell
def did_buy(df_period, buy_price,i):
    return [row1 if row1 <= buy_price else False for row1 in df_period]

# list comprehension for either profit or loss for each trigger aka WOL for win_or_loss
def wol(df_period, profit, loss, i):
    for row in df_period:
        if row >= profit*row:
            return 'profit'
        elif row <= loss*row:
            return 'loss'

In [13]:
sp500 = ["AMD", "BAC", "C", "CCL", "LUMN", "F", "FCX", "GE", "INTC", "MRO", "MSFT", "OXY", "PFE", "T", "VZ", "WFC", "WMB", "XOM", "CSCO", "UAL", "DAL", "TSLA", "GM", "FB", "NCLH", "AAL", "AMCR"] 
symbols = sp500[0:2]
symbols = ["APA", "IVZ", "DVN", "FTI", "GE", "HAL", "HBAN", "MRO", "NOV", "OXY", "NLOK", "NLSN", "AAL", "HPE"]
symbols = ["CRIS", "TZPC", "LQDT", "ONCT", "RENN", "PERI", "BIIO", "FUV", "SYRS", "YALA", "ESEA", "ARLO", "CANG", "HOLI", "KLXE", "CNDT", "X", "CMPI", "ASPN", "ARAV", "CMRX", "CPE", "FCEL", "RAD", "SWKH", "SQBG", "MEC", "HOOK", "SM", "FLUX", "RMG", "FFHL"]
symbols=["CRIS", "TZPC", "IGMS", "LQDT", "SFIX", "NKTX", "ONCT", "RENN", "PERI", "EH", "CMD", "FMTX", "BIIO", "FUV", "SYRS", "CTRA", "YALA", "PLUG", "GOGO", "ESEA", "FSLY", "ARLO", "CANG", "PASG", "BEEM", "KLXE", "CNDT", "SYX", "CVAC", "GTHX", "ASPN", "KYMR", "CMRX", "FCEL", "IMKTA", "SQBG", "BEAM", "SMAR", "VLDR", "RCUS", "SM", "EDIT", "FLUX", "RMG", "FFHL", "LMPX"]
symbols = ["LAZR", "ARCT", "BEAM", "MRNA", "FATE", "MSTR", "CIIC", "SNOW", "CRSP", "NTLA", "SPCE", "CVAC", "EDIT", "NVAX", "KSS", "PD", "SPOT", "ZS", "NET", "TSLA", "NNOX", "U", "IGMS", "ACIW", "XPO", "SMAR", "LYFT", "BNTX", "WYNN", "JHG", "PHR", "CRWD", "RDFN", "APPN", "DLTR", "BILI", "LNC", "NYT", "BBL", "PVH", "CKH", "BHP", "LRCX", "MU", "PFE", "NTAP", "IRM", "BA", "PKI", "SAIL", "TWST", "STM", "ETSY", "CZR", "QRVO", "EXAS", "RIO", "AEM", "XLNX", "ENB", "SWKS", "STX", "FTCH", "CBRE", "AMAT", "SHOP", "YNDX", "SKX", "BIGC", "JMIA", "BYD", "AMD", "MCHP", "CCEP", "CHGG", "GDDY", "AIV", "QCOM", "LITE", "WDAY", "PAAS", "FTNT", "DT", "CTXS", "SNAP", "DAL", "HCAT", "OHI", "DDOG", "IIVI", "APO", "AVGO", "AAPL", "GRWG", "SE", "PYPL", "BIDU", "ROKU", "TSM", "INTU"]
symbols = ["CCL", "AAL", "NCLH", "WYNN", "OXY", "BKR", "LNC", "MGM", "PVH", "APA", "LRCX", "MU", "PFE", "NTAP", "IRM", "BA", "PKI", "FCX", "ETSY", "QRVO", "HPQ", "SWKS", "STX", "CBRE", "BEN", "AMAT", "FTI", "IVZ", "AMD", "MCHP", "DVN", "AIV", "QCOM", "HAL", "FTNT", "CTXS", "DAL", "HPE", "DXC", "AVGO", "AAPL", "PYPL", "MRO", "INTU", "ATVI", "NFLX", "TER", "MAR", "TAP", "IPG", "ICE", "RCL", "STT", "HOLX", "EA", "TTWO", "CDNS", "ADBE", "CF", "NLSN", "KEYS", "MS", "TJX", "COF", "SYY", "FB", "VIAC", "SYF", "BAC", "CFG", "TMO", "SLB", "TXN", "AAP", "DFS", "MXIM", "PXD", "NWL", "DIS", "FITB", "OMC", "HES", "HWM", "TEL", "ROST", "FLIR"]
# symbols = symbols[70:]
symbols = ["WDLF", "SFOR", "CRIS", "BLSP", "RSCF", "ADTM", "WCVC", "RETC", "APTY", "WSGF", "ARST", "PUGE", "GLCO", "AQB", "GRNH", "PLG", "HYSR", "ADGO", "MAXD", "RCHA", "RTON", "RGBP", "INQD", "ONCT", "EVIO", "LXRX", "JAGX", "EPAZ", "STWC", "SNPW", "TRXC", "TRCH", "PLPL", "BCRX", "SINO", "NTRR", "RAD", "AMPE", "AEXE", "KODK", "CBBT", "NURO", "HTZGQ", "GRSO", "IMTV", "BB", "EDIT", "CIDM", "SESN", "CPE", "RIOT", "JCPNQ", "NNDM", "ADHC", "INFI", "X", "TLSS", "QEP", "PDGO", "AEPT", "OCLG", "JPEX", "FTXP", "AIKI", "UMC", "ARPO", "ACST", "CTXR", "SNMN", "JWN", "PSTI", "BSRC", "NCLH", "CDEV", "SWRM", "TLGT", "ONTX", "CLF", "GTE", "GNUS", "AAL", "FUBO", "VTMB", "USIO", "SRNE", "CCL", "MVIS", "MT", "VALE", "SM", "TNXP", "SAVE", "RCON", "FCEL", "HIPH", "VSYM", "CZNI"]
symbols = ["ARCS", "SFOR", "WDLF", "WSGF", "CRIS", "GTCH", "MFST", "WCVC", "XBIO", "GRSO", "ONCT", "DCLT", "GOVX", "UCPA", "SDVI", "ASTI", "BNGI", "RMRK", "MAXD", "LXRX", "EPAZ", "SLS", "SBES", "LGBS", "CIDM", "PLG", "SRCO", "RSCF", "AHT", "AQB", "CWIR", "DKAM", "BMIX", "TRCH", "TLSS", "BCRX", "INQD", "GENE", "VRUS", "CDEV", "RAD", "JCPNQ", "RSPI", "QEP", "TLGT", "BLSP", "CBTC", "ADTM", "NTRR", "ALKM", "SM", "CPE", "GRCU", "RGLS", "CNET", "TRXC", "HYSR", "VERU", "BOTY", "BMXC", "TMXN", "GTE", "AMPE", "PUPS", "OXY", "CLSN", "DNN", "ALSK", "MIK", "VGTL", "DEWM", "STWC", "CLSK", "FMCC", "BLNK", "MRO", "MAC", "MLFB", "UMC", "X", "XTRM", "PDOS", "NCLH", "CNDT", "ACRL", "AAL", "MUR", "PNAT", "APA", "PZOO", "MSRT", "BANT", "CCJ", "SAVE", "LPTX", "GOGO", "BLXX", "PE"]
symbols = ["MRO", "AAL", "APA", "GE", "FTI", "HPE"]
symbols = ["ADMA", "ALSK", "AHT", "INSG", "PUPS", "KATX", "CLTS", "ONTX", "MSRT", "CBBT", "GRCU", "SM", "SRNE", "KSS", "CLSK", "FUBO", "IGT", "MEDH", "NCLH", "GTE", "TNXP", "GNUS", "OPK", "NNDM", "JWN", "OXY", "MVIS", "TECK", "BTDG", "BKR", "MLFB", "BLXX", "VSYM", "ALKM", "CNDT", "SAVE", "CLF", "SAN", "SID", "BBVA", "VALE", "AGI", "MAC", "CDE", "MRO", "TGRR", "FNMA", "FUNN", "VET", "INO", "TAUG", "SHIP", "CCL", "EVUS", "ADXS", "MUR", "HAL", "TEVA", "PRPO", "MRMD", "REI", "GE", "OVV", "ACRL", "AUY", "CVE", "FTI", "HPE", "TPTW", "CZNI", "EGO", "ITUB", "BLNK", "IBN", "THC", "ON", "CBKC", "GNW", "CELZ", "APA", "APPS", "NOV", "M", "AVGR", "SLB", "DLOC", "CLDR", "MGM", "ENB", "PBR"]


symbols = ["BB", "ZNGA", "CCJ", "VALE", "FCAU", "RAD", "DBX", "GPS", "EGHT", "MU", "ATVI", "SBUX", "ABBV", "BIDU", "LLY"]
# symbols = symbols[32:]
symbols = ["SFOR", "WDLF", "TMXN", "MAXD", "SWRM", "AEPT", "DIGAF", "NECA", "GRSO", "RCHA", "BCDA", "VGTL", "WHEN", "IFXY", "ADTM", "RGBP", "TLSS", "TSPG", "ASTI", "NTRR", "SHMP", "ADHC", "VSYM", "CELZ", "MVIS", "INQD", "EVUS", "DS", "VRUS", "JPEX", "FTXP", "BLSP", "OXY", "UNGS", "KWBT", "BLNK", "GAHC", "GNUS", "MGTI", "HMNY", "RIG", "SIOX", "IPSI", "MT", "AAL", "SAVE", "HYSR", "PLUG", "RIOT", "BANT", "GRST", "MARA", "DTEA", "ET", "BIEL", "BTCS", "IFAN", "XOM", "NCLH", "GE", "NPHC", "BP", "CCL", "WFC", "HPQ"]
# symbols = symbols[47:]
symbols = ["SFOR", "MAXD", "ARCS", "NECA", "CRIS", "EVUS", "TMXN", "USEI", "SWRM", "CBTC", "RCHA", "IFAN", "CRSM", "AMPE", "BSRC", "LXRX", "ASTI", "HYGN", "ADTM", "DRNG", "SHMP", "WSGF", "VUZI", "GRPS", "MVIS", "GRSO", "TINO", "VGTL", "AEPT", "SLS", "MARA", "ACRL", "BTCS", "GEGI", "ONCT", "TSPG", "BTU", "CELZ", "EPAZ", "DIGAF", "CLSK", "DS", "TLSS", "TLGT", "BLNK", "CYIO", "ICLD", "UAVS", "MCET", "ADGO", "IMTV", "RMRK", "JPEX", "PBYA", "UEC", "SEEL", "AMRS", "FUBO", "SGMO", "MGTI", "BYOC", "SNCA", "BCRX", "LBSR", "SOL", "DLOC", "BCDA", "AHT", "NTRR", "UUUU", "INSG", "SFRX", "JCPNQ", "MNKD", "SRMX", "FPFI", "HMNY", "UNGS", "WCVC", "GEVO", "BONZ", "ADSV", "AR", "KATX", "DNN", "SING", "ELYS", "BANT", "GHHC", "CLNE", "BLXX", "LPTX", "INFN", "KODK", "PLUG", "GBTC", "DPLS", "MRPI", "WDLF"]
symbols = ["MRPI", "ABML", "FDBL", "WPUR", "OCGN", "BSRC", "EVUS", "UAVS", "DRNG", "MVIS", "AQMS", "DSGT", "DKAM", "ZONX", "USEI", "MJTK", "ABWN", "MARA", "CBBT", "VUZI", "ICLD", "DSCR", "AMPE", "SHMP", "CLNE", "BLNK", "ENRT", "MMMW", "KOPN", "FEYE", "AMRS", "GEVO", "BBRW", "GHHC", "SOL", "MGNI", "EMAN", "RIOT", "VISM", "GME", "NNDM", "VISL", "PTN", "GAXY", "CLRB", "ONTX", "OCLG", "FCEL", "CELZ", "LBSR", "GRSO", "CBAT", "TKOI", "PLUG", "CZNI", "BMIX", "WWR", "OPTT", "SPWR", "MARK", "CBTC", "INSG", "PERI", "CLDR", "GSAT", "BLDP", "APTY", "ZOM", "MBIO", "XELA", "HYSR", "RBNW", "WOFA", "ALRN", "MGI", "CRBP", "VBIV", "ENG", "SING", "WKHS", "FTEK", "COTY", "IGEN", "AVGR", "NAK", "WATT", "NRZ", "CHEK", "OGEN", "VVPR", "SRNE", "ALYI"]
# symbols = symbols[25:]
# sp500 = ["HWM", "ADM", "AES", "AFL", "AIG", "APA", "IVZ", "BAC", "TFC", "BEN", "BKR", "BK", "BSX", "BWA", "CAG", "CCL", "CNP", "COG", "TPR", "COP", "CPB", "DXC", "LUMN", "DISH", "DRE", "DVN", "EOG", "EXC", "F", "FAST", "FCX", "FE", "FITB", "FTI", "GE", "GLW", "GPS", "HAL", "HBAN", "PEAK", "HIG", "HST", "HPQ", "HRL", "INTC", "IP", "IPG", "IRM", "JNPR", "KEY", "KIM", "KR", "LKQ", "LNC", "LB", "LUV", "MET", "MO", "MOS", "MRO", "VTRS", "NI", "NOV", "NRG", "NWL", "OKE", "OXY", "PFE", "PFG", "PHM", "PPL", "PRGO", "REG", "RF", "ROL", "T", "SLB", "NLOK", "JCI", "UDR", "UNM", "USB", "VNO", "VTR", "WBA", "WDC", "WFC", "WMB", "WY", "XOM", "XRX", "ZION", "TAP", "HFC", "CSCO", "MGM", "DISCA", "CF", "UAA", "VIAC"]
# symbols = sp500
symbols = ["WWR", "HCHC", "TEVA", "SEEL", "MBRX", "CVE", "DS", "STL", "AGNC", "HMPQ", "SDPI", "ITOX", "VXRT", "ORCL", "BTG", "VMSI", "BBI", "HAL", "SLB", "SM", "WBA", "TXTM", "AG", "ADHC", "BBBY", "CVX", "BMY", "SAVE", "OPK", "GPORQ", "INSG", "DIS", "POAI", "CDE", "GILD", "VUZI", "AMRS", "YRLLF", "TRVN", "CRBP", "JWN", "CMCSA", "ADMA", "ACRL", "TTCM", "MS", "SESN", "DVN", "TRNX", "TXMD", "TWO", "OPTT", "SPWH", "SU", "TSPG", "RDS.A", "CLRB", "AQB", "CWIR", "QRTEA", "DPW", "TMXN", "AEG", "WMT", "MARK", "MTNB", "ENSV", "DQ", "LUV", "PCG", "KNDI", "MIK", "PACB", "SABR", "DVAX", "NGD", "AIKI", "XTRM", "GOGO", "KSS", "RWLK", "TGHI", "HSDT"]
symbols = ['NCLH']

In [17]:
days = 10
c = pd.DataFrame({})
sym_dict = []



for i,symbol in enumerate(symbols):
    
    
    # Get the price history iteratively
    data = TDSession.get_price_history(
        symbol, 
        period_type=Client.PriceHistory.PeriodType.DAY,
        period=Client.PriceHistory.Period.TEN_DAYS,
        frequency_type=Client.PriceHistory.FrequencyType.MINUTE,
        frequency=Client.PriceHistory.Frequency.EVERY_MINUTE)
    data = data.json()

    
    # Create dataframe with new features
    test = pd.DataFrame(data['candles'])
    test['time'] = test.apply(lambda row: datetime.fromtimestamp(row.datetime/1000).strftime("%I:%M:%S"),axis=1 )
    test['day'] = test.apply(lambda row: datetime.fromtimestamp(row.datetime/1000).strftime("%d"),axis=1 )
    test['month'] = test.apply(lambda row: datetime.fromtimestamp(row.datetime/1000).strftime("%B"),axis=1 )
    test['year'] = test.apply(lambda row: datetime.fromtimestamp(row.datetime/1000).strftime("%Y"),axis=1 )
    #create open to high column
    test['open_high'] = test['high']-test['open']
    #create open to low column
    test['open_low'] = test['open']-test['low']
    test

    
    # Define the conditions for the trigger
    #define variables
    candle_diff = 1.2
    morn15 = ['08:{}:00'.format(30+x) for x in range(0,15)]
    
    # recent low mean
    recent_low_mean = test.iloc[-600:,2].mean()

    # past high mean
    past_high_mean = test.iloc[0:600,1].mean()

    #define open higher than open low
    high_not_zero = test['open_high'] != 0 
    low_not_zero = test['open_low'] != 0 
    tall_candle = test['open_high'] > candle_diff*test['open_low']
    volume75 = test['volume'] >= np.percentile(test['volume'],[70])[0] #90
    not_first15 = ~test['time'].isin(morn15)
    price65th = test['open']<np.percentile(test['high'],[65])[0] #65
    down_trend = past_high_mean < recent_low_mean * 1.02
    price35th = test['open']>np.percentile(test['high'],[45])[0]

    condition = high_not_zero & low_not_zero & tall_candle & volume75 & not_first15 & price65th#  & price35th & down_trend

    
    # Checking for trigger, store in trigger_df
    trigger_df = test.loc[condition, :].reset_index()

    
    #add column of desired sell prices
    #desired roi is 0.5%
    my_profit = 1.0019
    trigger_df['to_sell_price'] = trigger_df.open * my_profit
    trigger_df

    sell_dur_min = 5
    ## Bought
    #the time period that you want to get after the trigger
    time_period6 = [get_period(6, row[6], test) for i,row in trigger_df.iterrows()]

    #did we buy???
    trigger_df['bought'] = pd.DataFrame([did_buy(time_period6[i].low,row[1],i) for i,row in trigger_df.iterrows()]).any(axis=1)

    
    ## Sold
    #the time period that you want to get after the trigger
    time_period20 = [get_period(sell_dur_min, (row[6]), test) for i,row in trigger_df.iterrows()]

    #did we sell???
    sold = pd.DataFrame([did_sell(time_period20[i].low,row[13],time_period20[i].datetime) for i,row in trigger_df.iterrows()])
    #sold_time = [did_sell(time_period20[i].low,row[13],time_period20[i].datetime) for i,row in trigger_df.iterrows()][0][1]
    trigger_df['sold'] = pd.DataFrame(sold).any(axis=1)


    ## Lost
    #the time period that you want to get after the trigger
    time_period20 = [get_period(sell_dur_min, (row[6]+60000), test) for i,row in trigger_df.iterrows()]

    #did we lose???
    trigger_df['lost'] = pd.DataFrame([did_buy(time_period20[i].low,(row[1]*.10),i) for i,row in trigger_df.iterrows()]).any(axis=1)

    trigger_df['lost'].sum()
    trigger_df.loc[trigger_df['lost']==False,:]

    trigger_df.day.value_counts().sort_index()

    profit = (trigger_df['bought'] == True) & (trigger_df['sold'] == True)
    len(trigger_df.loc[profit,:])

    loss = (trigger_df['bought'] == True) & (trigger_df['sold'] == False)
    
    no_buy = (trigger_df['bought'] == False)
#     print(trigger_df.loc[loss,:])

    min_taken = [(test.loc[(test['high']>=x[13])&(test['datetime']>x[6]), ['datetime']].iloc[0]-x[6])/60000 if test.loc[(test['high']>=x[13])&(test['datetime']>x[6]), 'datetime'].any() == True else 9999999 for i,x in trigger_df.loc[profit|loss|no_buy,:].iterrows()]
#     min_df = trigger_df.drop(trigger_df.loc[trigger_df['bought']==False,:].index) 
    min_df = trigger_df.copy()
    min_df['min_taken'] = min_taken
    min_df.dtypes
#     min_df.sort_values('min_taken', ascending=False)
    
#     trigger_df.loc[loss,:]
    #trigger_df.loc[profit,:]
    sym_data = {
        'symbol':symbol,
        'total_sells':len(trigger_df.loc[profit,:]),
        'total_loss':len(trigger_df.loc[loss,:]),
        'min_to_sell':round(min_df.min_taken.mean(),4),
        'profit': my_profit,
        'sold': trigger_df.sold.any().sum(),
        'lost': trigger_df.lost.any().sum()
    }

    sym_dict.append(sym_data)
    print(symbol,i)

trades = pd.DataFrame.from_dict(sym_dict, orient='columns')

trades

NCLH 0


,symbol,total_sells,total_loss,min_to_sell,profit,sold,lost
0,NCLH,68,228,70136.5397,1.0019,1,0


In [16]:
trigger_df

,index,open,high,low,close,volume,datetime,time,day,month,year,open_high,open_low,to_sell_price,bought,sold,lost
0,1214,25.2981,25.3199,25.2800,25.3199,86676,1607978520000,02:42:00,14,December,2020,0.0218,0.0181,25.346166,True,False,False
1,1215,25.3100,25.3400,25.2850,25.3058,109286,1607978580000,02:43:00,14,December,2020,0.0300,0.0250,25.358089,True,False,False
2,1218,25.2550,25.2999,25.2400,25.2999,136415,1607978760000,02:46:00,14,December,2020,0.0449,0.0150,25.302985,False,True,False
3,1219,25.2900,25.3400,25.2800,25.3400,69797,1607978820000,02:47:00,14,December,2020,0.0500,0.0100,25.338051,False,True,False
4,1220,25.3300,25.3700,25.3200,25.3250,117683,1607978880000,02:48:00,14,December,2020,0.0400,0.0100,25.378127,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,6234,24.3886,24.4200,24.3701,24.4100,30414,1608824820000,09:47:00,24,December,2020,0.0314,0.0185,24.434938,False,True,False
424,6343,24.4600,24.5100,24.4511,24.5100,52943,1608831360000,11:36:00,24,December,2020,0.0500,0.0089,24.506474,True,False,False
425,6354,24.4300,24.4689,24.4250,24.4600,30389,1608832020000,11:47:00,24,December,2020,0.0389,0.0050,24.476417,True,True,False
426,6361,24.5400,24.5700,24.5350,24.5600,33800,1608832440000,11:54:00,24,December,2020,0.0300,0.0050,24.586626,True,False,False


In [31]:
trades.sort_values('min_to_sell', ascending=True).head(29)

,symbol,total_sells,total_loss,min_to_sell,profit
15,VMSI,2,0,1.2500,1.0019
39,CRBP,89,2,1.3738,1.0019
81,TGHI,1,0,2.0000,1.0019
49,TXMD,113,0,2.1721,1.0019
3,SEEL,96,7,2.8657,1.0019
56,CLRB,98,0,3.3481,1.0019
66,ENSV,39,1,4.6140,1.0019
82,HSDT,5,0,5.4118,1.0019
75,NGD,77,3,6.9072,1.0019
57,AQB,196,4,8.8076,1.0019


In [32]:
trades.sort_values('total_loss', ascending=True).sum()

symbol         HSDTTTCMYRLLFAMRSVUZITXMDDISGPORQTSPGCLRBCWIRA...
total_sells                                                 8119
total_loss                                                   511
min_to_sell                                          6.66146e+06
profit                                                   83.1577
dtype: object

In [33]:
trades.loc[trades['min_to_sell']<6,:].symbol.values

array(['SEEL', 'VMSI', 'CRBP', 'TXMD', 'CLRB', 'ENSV', 'TGHI', 'HSDT'],
      dtype=object)

In [34]:
trades.loc[(trades['total_loss']==0)&(trades['min_to_sell']<10),:].symbol.values

array(['VMSI', 'GPORQ', 'TXMD', 'CLRB', 'TGHI', 'HSDT'], dtype=object)

In [35]:
trades.loc[(trades['total_loss']==0)&(trades['min_to_sell']<12),:].sum()

symbol         VMSIADHCGPORQTXMDCLRBMARKTGHIHSDT
total_sells                                  291
total_loss                                     0
min_to_sell                              44.9815
profit                                    8.0152
dtype: object

In [95]:
trades.loc[(trades['total_loss']==0),:].sum()

symbol         AFLBACDVNLUVRFUSBMGM
total_sells                    1306
total_loss                        0
min_to_sell                 61.4598
profit                       7.0021
dtype: object

In [96]:
trades.loc[(trades['total_loss']==0),:].count()

symbol         7
total_sells    7
total_loss     7
min_to_sell    7
profit         7
dtype: int64

In [97]:
trades.loc[:,'profit'].iloc[0]

1.0003